## VISUALIZATION of intrahemisphere connectivity

Similar to corpus callosum. Visualize relationships between extracted MRI variables and cognitive values.

In [27]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from statannot import add_stat_annotation
import glob
import statsmodels.api as sm
import statsmodels.formula.api as smf
from functools import reduce

In [28]:
# Load the csv

csv_cc = 'cc.csv'
# csv_hemis = 'graph_intrainter_cort_G_SC.csv'
csv_hemis = 'graph_dti_G_SC_cort.csv'
csv_hemis_FC = 'graph_G_FC.csv'
csv_hemis_cl = 'graph_CLINIC_og_G_SC.csv'
csv_hemis_nodes = 'graph_nodes_SC.csv'
csv_total = 'data_total.csv'

df_cc = pd.read_csv(csv_cc)
df_hemis = pd.read_csv(csv_hemis)
df_hemis_FC = pd.read_csv(csv_hemis_FC)
df_hemis_cl = pd.read_csv(csv_hemis_cl)
df_hemis_nodes = pd.read_csv(csv_hemis_nodes)
df_total = pd.read_csv(csv_total)

In [ ]:
# merge results and total
data_frames = [df_total, df_cc, df_hemis, df_hemis_FC]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=["SubjID", "CENTER"]), data_frames)
df_merged = df_merged[df_merged.QC == "Y"]

data_frames = [df_total, df_cc, df_hemis_cl]
df_merged_cl = reduce(lambda  left,right: pd.merge(left,right,on=["SubjID", "CENTER"]), data_frames)
df_merged_cl = df_merged_cl[df_merged_cl.QC == "Y"]


print(df_merged["GROUP"].describe())
df_merged.head()

In [ ]:
## Boxplots center and diagnosis
## Do for all the CC values
df_merged["Full_hemis"] = ( df_merged["SC_L_avg_spl"] + df_merged["SC_R_avg_spl"] ) / 2
df_merged["FC_spl_full"] = ( df_merged["FC_L_avg_spl"] + df_merged["FC_R_avg_spl"] ) / 2
df_merged["FC_eff_full"] = ( df_merged["FC_L_efficiency"] + df_merged["FC_R_efficiency"] ) / 2

list_of_values = ["Full_hemis", "SC_avg_spl", "FC_spl_full", "FC_inter_avg_spl", "FC_eff_full", "FC_inter_efficiency"]

for cc in list_of_values:

    fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(15,5))
    plt.suptitle(f"{cc}: boxplots", y=1.0)
    sns.boxplot(data=df_merged, x="GROUP", y=f"{cc}", ax=ax1)
    sns.swarmplot(data=df_merged, x="GROUP", y=f"{cc}", ax=ax1, color=".2", size=3)

    sns.boxplot(data=df_merged, x="CENTER", y=f"{cc}", ax=ax2)
    sns.swarmplot(data=df_merged, x="CENTER", y=f"{cc}", ax=ax2, color=".2", size=3)

In [ ]:
## REGRESSION
# Compare each CC to SDMT and EDSS. 
from scipy import stats

variable_to_plot = "Test" # needs to change every time

def annotate(data, **kws):
    r, p = stats.pearsonr(data[variable_to_plot], data[f"{cc}"])
    ax = plt.gca()
    ax.text(.05, .8, 'r={:.2f}, p={:.2g}'.format(r, p),
            transform=ax.transAxes)


list_of_values = ["Full_hemis", "SC_inter_avg_spl", "FC_spl_full", "FC_inter_avg_spl", "FC_eff_full", "FC_inter_efficiency"]
for cc in list_of_values:

    for variable_to_plot in ["AGE", "DD", "SDMT", "EDSS"]:

        data_to_plot = df_merged
        if variable_to_plot in ["DD", "SDMT", "EDSS"]:
            data_to_plot = df_merged[df_merged.GROUP != "HC"].dropna(subset=[variable_to_plot])

        # AGE
        g = sns.lmplot(data=data_to_plot, x=variable_to_plot, y=f"{cc}", col="GROUP", hue="GROUP")
        g.map_dataframe(annotate)



In [ ]:
def annotate(col1, col2, data, multihue=False, **kwargs):
    if len(data[col1]) >= 2 or len(data[col2]) >= 2:
        r, p = stats.pearsonr(data[col1], data[col2])
        ax = plt.gca()

        # hardcoded, udpate if working with other labels
        x_pos = 0.05
        if multihue:
            label = kwargs["label"]
            dict_of_labels = {"CIS": 0, "RRMS": 1, "PMS": 2, 'MS':0, 'HC':1}
            y_pos = 0.8 - dict_of_labels[kwargs["label"]]*0.05
        else:
            label =  " "
            y_pos = 0.8
        
        ax.text(x_pos, y_pos, f'{label}, ' + 'r={:.2f}, p={:.2g}'.format(r, p),
                transform=ax.transAxes)

# Sum all values of CC and divide by ICV
# average intrahemis connectivity
# plot one against the other

list_of_values = ["Full_hemis", "SC_inter_avg_spl", "FC_spl_full", "FC_inter_avg_spl", "FC_eff_full", "FC_inter_efficiency"]

df_merged["Full_CC"] = df_merged["CC_Posterior"] + df_merged["CC_Mid_Posterior"] + df_merged["CC_Central"] + df_merged["CC_Mid_Anterior"] + df_merged["CC_Anterior"]
df_merged["Comm_ratio_approx"] = ( df_merged["CC_Posterior"] + df_merged["CC_Mid_Posterior"] + df_merged["CC_Central"] + df_merged["CC_Mid_Anterior"] + df_merged["CC_Anterior"] ) / df_merged["EstimatedTotalIntraCranialVol"]
df_merged["CC_ratio_area"] = np.sqrt(df_merged["CC_Sag_area"]) / (df_merged["BrainSegVol"]**(1./3.))

# both 
df_merged["Full_hemis"] = ( df_merged["SC_L_avg_spl"] + df_merged["SC_R_avg_spl"] ) / 2
df_merged_cl["Full_hemis"] = ( df_merged_cl["SC_L_avg_spl"] + df_merged_cl["SC_R_avg_spl"] ) / 2

results = smf.ols(f'Full_hemis ~ C(CENTER)', data=df_merged).fit()
df_merged[f"Full_hemis_C"] = results.resid

results = smf.ols(f'Comm_ratio ~ C(CENTER)', data=df_merged).fit()
df_merged[f"Comm_ratio_C"] = results.resid

variable_to_plot = "Comm_ratio"
cc = "Full_hemis"
df_merged['disease'] = np.where((df_merged['GROUP']== 'HC'), "HC", "MS")
df_merged_hc = df_merged[df_merged.disease == "HC"]
g = sns.lmplot(data=df_merged, x=variable_to_plot, y=f"{cc}", col="CENTER")
g.map_dataframe(annotate, variable_to_plot, cc, multihue=False)


In [ ]:
df_merged["CC_Sag_area_sqrt"] = np.log10(np.sqrt(df_merged["CC_Sag_area"]))
df_merged["TIV_cubicroot"] = np.log10(df_merged["BrainSegVol"]**(1./3.))

g = sns.lmplot(data=df_merged, x="Comm_ratio", y="TIV_cubicroot", hue="disease")
g.map_dataframe(annotate, "Comm_ratio", "TIV_cubicroot", multihue=True)


In [ ]:
g = sns.lmplot(data=df_merged, x="TIV_cubicroot", y="CC_Sag_area_sqrt")

In [ ]:
variable_to_plot = "Full_hemis"
cc = "Full_hemis_mine"
df_merged_cl['Full_hemis_mine'] = df_merged[df_merged.CENTER=="CLINIC"]["Full_hemis"]
g = sns.lmplot(data=df_merged_cl, x=variable_to_plot, y=f"{cc}", robust=True)
g.map_dataframe(annotate)
